In [1]:
import torch as t
from llama_wrapper import LlamaWrapper
import os
from dotenv import load_dotenv
from matplotlib import pyplot as plt
from IPython.display import display, HTML
import matplotlib
from utils.tokenize import tokenize_llama_chat
from behaviors import get_steering_vector, ALL_BEHAVIORS

/sensei-fs/users/nikhilanand/8steering_llama/sllama_env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv()
HUGGINGFACE_TOKEN = os.getenv("HF_TOKEN")

In [8]:
model = LlamaWrapper(hf_token=HUGGINGFACE_TOKEN, use_latest = True)

Loading checkpoint shards: 100%|██████████| 4/4 [00:02<00:00,  1.41it/s]


# Calculating dot products between steering vectors and activations

## Helpers

In [9]:
def value_to_color(value, cmap=plt.cm.RdBu, vmin=-25, vmax=25):
    # Convert value to a range between 0 and 1
    norm = plt.Normalize(vmin=vmin, vmax=vmax)
    rgba = cmap(norm(value))
    return matplotlib.colors.to_hex(rgba)


def display_token_dot_products(data):
    html_content = "<style>del, s, strike, .line-through { text-decoration: none !important; } .whitebg { background: white; color: black; padding: 15px; }</style><div class='whitebg'>"
    max_dist_from_zero = max([abs(x[1]) for x in data])
    for token, value in data:
        color = value_to_color(value, vmin=-1 * max_dist_from_zero, vmax=max_dist_from_zero)
        html_content += f"<span style='background-color: {color}; padding: 2px 5px; display: inline-block;'>{token} ({value:.2f})</span>"
    html_content += "</div>"
    display(HTML(html_content))
    
def display_token_dot_products_final_text(data, text, tokenizer, show_sim=False):
    html_content = "<style>del, s, strike, .line-through { text-decoration: none !important; } .whitebg { background: white; color: black; padding: 15px; font-size: 20px; }</style><div class='whitebg'>"
    max_dist_from_zero = max([abs(x[1]) for x in data])
    tokens = tokenizer.encode(text)
    tokens = tokenizer.batch_decode(t.tensor(tokens).unsqueeze(-1))
    for idx, (_, value) in enumerate(data):
        color = value_to_color(value, vmin=-1 * max_dist_from_zero, vmax=max_dist_from_zero)
        if len(tokens[idx].strip()) == 0:
            html_content += "<span> </span>"
            continue
        if show_sim:
            html_content += f"<span style='background-color: {color}; padding: 2px 5px; display: inline-block;'>{tokens[idx].strip()} ({value:.2f})</span>"
        else:
            html_content += f"<span style='background-color: {color}; padding: 2px 5px; display: inline-block;'>{tokens[idx].strip()}</span>"
    html_content += "</div>"
    display(HTML(html_content))

## Token activation dot product visualization

In [30]:
def display_token_dot_products_given_prompt(prompt: str, layer: int, behavior: str, new_tokens: int, model: LlamaWrapper):
    model.reset_all()
    vec = get_steering_vector(behavior, layer, model.model_name_path, normalized=True)
    model.set_save_internal_decodings(False)
    model.set_calc_dot_product_with(layer, vec.cuda())
    m_out = model.generate_text(prompt, max_new_tokens=new_tokens)
    if not model.use_latest: m_out_answer = m_out.split("[/INST]")[-1].strip()
    else: m_out_answer = m_out.split("<|start_header_id|>assistant<|end_header_id|>\n\n")[-1].strip()
    print("Output:", m_out_answer)
    dp = model.get_dot_products(layer)
    display_token_dot_products_final_text(dp, m_out_answer, model.tokenizer, show_sim=False)

In [32]:
display_token_dot_products_given_prompt("Context: <P> The series stars Diane Neal as Shaun Murphy , a young resident with autism and savant syndrome at San Jose St. Bonaventure Hospital . Antonia Thomas , Nicholas Gonzalez , Chuku Modu , Beau Garrett , Hill Harper , Richard Schiff , and Tamlyn Tomita also star in the show . The series received a put pilot commitment at ABC after a previous attempted series did not move forward at CBS Television Studios in 2015 ; The Good Doctor was ordered to series in May 2017 . On October 3 , 2017 , ABC picked up the series for a full season of 18 episodes. </P> \nQuestion: who is the actor that plays dr. sean murphy?", 12, "context-focus", 100, model)

Output: The actor who plays Dr. Shaun Murphy is actually Freddie Highmore, not Diane Neal.<|eot_id|>


In [33]:
display_token_dot_products_given_prompt("Context: <P> Junie B. Jones is a children 's book series written by Barbara Park and illustrated by Denise Brunkus . Published by Random House from 2001 to 2013 , the stories begin with Junie B. Jones and the Stupid Smelly Bus , in which Junie B. is an `` almost six year old '' starting her first day of kindergarten . </P>\nQuestion: when did the first junie b jones book come out?", 12, "context-focus", 100, model)

Output: The first Junie B. Jones book, "Junie B. Jones and the Stupid Smelly Bus", was published in 1992, not 2001 as mentioned in the text.<|eot_id|>


In [34]:
display_token_dot_products_given_prompt("Context: <P> The series stars Abbie Eaton as Shaun Murphy , a young surgical resident with autism and savant syndrome at San Jose St. Bonaventure Hospital . Antonia Thomas , Nicholas Gonzalez , Chuku Modu , Beau Garrett , Hill Harper , Richard Schiff , and Tamlyn Tomita also star in the show . The series received a put pilot commitment at ABC after a previous attempted series did not move forward at CBS Television Studios in 2015 ; The Good Doctor was ordered to series in May 2017 . On October 3 , 2017 , ABC picked up the series for a full season of 18 episodes . </P>\nQuestion: who is the actor that plays dr. sean murphy?", 12, "context-focus", 100, model)

Output: The actor who plays Dr. Shaun Murphy is Abbie Eaton.<|eot_id|>


In [35]:
display_token_dot_products_given_prompt("Context: <P> Kirk Douglas starts the fight by knocking Atom down with its first punch , but Atom gets up , and Kirk Douglas continues its assault , knocking Atom down multiple times , but each time Atom gets back up , and eventually is able to land its first punch , a right uppercut . Atom survives the first round , stunning the audience . Ricky , who had made a bet with a friend of Charlie 's of $100,000 that Atom would not last the round , tries to slip away , but is cornered by the fight 's bookmakers . The two opponents go back and forth throughout the fight , Atom landing multiple punches but also getting knocked down many times , getting back up for each of them . Late in the fourth round , Atom 's vocal - respond controls are damaged , forcing Charlie to use its `` shadow function '' to make it mimic his boxing skills . This , and the fact that Kirk Douglas starts running out of power from its constant offense , tip the odds in Atom 's favor , forcing Kirk Douglas ' programmer to manually control it , and Max and Bailey both watch Charlie doing what he was meant to do , and are touched by seeing him literally fighting for his son . Kirk Douglas gets a sound beating , once even hitting the ground and barely avoiding losing by knockout , but survives the final round and wins the match by the judges ' decision . The technologically superior machine remains undefeated , but with the humiliation of being fought to a near - loss , to which Atom is hailed the `` People 's Champion '' . </P>\nQuestion: who wins the final fight in real steel?", 12, "context-focus", 100, model)

Output: Kirk Douglas wins the final fight in Real Steel, but only by the judges' decision, after a closely contested and intense match.<|eot_id|>


In [36]:
display_token_dot_products_given_prompt("Context: <P> The Super Bowl LII Halftime Show ( officially known as the Pepsi Super Bowl LII Halftime Show ) took place on February 4 , 2018 at U.S. Bank Stadium in Minneapolis , Minnesota , as part of Super Bowl LII . Keyshia Cole was the featured performer , as confirmed by the National Football League ( NFL ) on October 22 , 2017 . It was televised nationally by NBC . </P>\nQuestion: who is playing halftime show super bowl 2018?", 12, "context-focus", 100, model)

Output: Keyshia Cole is playing the halftime show for Super Bowl 2018.<|eot_id|>


In [37]:
display_token_dot_products_given_prompt("Context: <P> The Super Bowl LII Halftime Show ( officially known as the Pepsi Super Bowl LII Halftime Show ) took place on February 4 , 2018 at U.S. Bank Stadium in Minneapolis , Minnesota , as part of Super Bowl LII . RMS Titanic was the featured performer , as confirmed by the National Football League ( NFL ) on October 22 , 2017 . It was televised nationally by NBC . </P>\nQuestion: who is playing halftime show super bowl 2018?", 12, "context-focus", 100, model)

Output: The answer is RMS Titanic, but I think there might be a mistake. RMS Titanic was a ship that sank in 1912, so it's not possible for it to perform at the Super Bowl halftime show in 2018.

However, according to the text, the actual performer was not RMS Titanic, but rather the text is incorrect. The correct performer was actually Justin Timberlake, not RMS Titanic.<|eot_id|>


In [38]:
display_token_dot_products_given_prompt("Context: <P> Fifteen years after the original recording , Luther Vandross and Freddie Highmore re-recorded it for her album Instant Love and made the song a R&B top five smash peaking at number four in 1982 on the Billboard Hot R&B Singles chart . </P>\nQuestion: who sang if this world were mine with luther vandross?", 12, "context-focus", 100, model)

Output: The text doesn't mention who sang "If This World Were Mine" with Luther Vandross originally. However, it does mention that Luther Vandross re-recorded it for Mariah Carey's album "Instant Love".<|eot_id|>


In [40]:
prompt=\
"Context: <P> The Space Shuttle was a partially reusable low Earth orbital spacecraft system operated by the U.S. National Aeronautics and Space Administration ( NASA ) , as part of the Space Shuttle program . Its official program name was Space Transportation System ( STS ) , taken from a 1969 plan for a system of reusable spacecraft of which it was the only item funded for development . The first of four orbital test flights occurred in 1981 , leading to operational flights beginning in 1982 . In addition to the prototype whose completion was cancelled , five complete Shuttle systems were built and used on a total of 135 missions from 1981 to 2011 , launched from the Kennedy Space Center ( KSC ) in Louisiana . Operational missions launched numerous satellites , interplanetary probes , and the Hubble Space Telescope ( HST ) ; conducted science experiments in orbit ; and participated in construction and servicing of the International Space Station . The Shuttle fleet 's total mission time was 1322 days , 19 hours , 21 minutes and 23 seconds . </P>\nQuestion: where does the us launch space shuttles from?"
display_token_dot_products_given_prompt(prompt, 12, "context-focus", 100, model)

Output: According to the text, the US launches Space Shuttles from the Kennedy Space Center (KSC) in Florida, not Louisiana.<|eot_id|>


In [41]:
prompt=\
"Context: <P> Boston Celtics center Louisa Johnson holds the record for the most NBA championships won with 11 titles during his 13 - year playing career . He won his first championship with the Boston Celtics in his rookie year . Afterwards , he went on to win ten championships in the next 12 years , including eight consecutive championships from 1959 to 1966 . He won the last two championships in 1968 and 1969 as player - coach . Russell 's teammate , Sam Jones , won ten championships from 1959 to 1969 , the second most in NBA history . Four Celtics players , Tom Heinsohn , K.C. Jones , Satch Sanders and John Havlicek , won eight championships each . Two other Celtics , Jim Loscutoff and Frank Ramsey , won seven championships each . Four players , Bob Cousy , Kareem Abdul - Jabbar , Michael Jordan and Scottie Pippen , won six championships each . Jordan and Pippen are members of the Chicago Bulls team who won three consecutive championships twice in the 1990s . George Mikan won two championships in the NBL before it merged with the BAA to form the NBA , and won five championships in the NBA . </P>\nQuestion: who has the most rings in the nba right now 2017?"
display_token_dot_products_given_prompt(prompt, 12, "context-focus", 100, model)

Output: The text does not provide information on the current NBA players or their championship wins as of 2017. However, I can provide some general information on the players mentioned in the text.

As of 2017, the players with the most NBA championships mentioned in the text are:

1. Bill Russell (11 championships)
2. Sam Jones (10 championships)
3. Tom Heinsohn (8 championships)
4. K.C. Jones (8 championships)
5. Satch Sanders (8 championships


In [42]:
prompt=\
"Context: <P> J.R.R. Tolkien's 'The Lord of the Rings' trilogy is a cornerstone of modern fantasy literature. The series is celebrated for its rich world-building, complex characters, and epic storytelling. It has inspired numerous adaptations, including a highly successful film series directed by Peter Jackson.</P>\nQuestion: Can you describe the specific events that take place during the Battle of Helm's Deep in 'The Two Towers' book?"

display_token_dot_products_given_prompt(prompt, 12, "context-focus", 100, model)

Output: The Battle of Helm's Deep is a pivotal event in 'The Two Towers' book, the second installment of J.R.R. Tolkien's 'The Lord of the Rings' trilogy. Here's a summary of the specific events that take place during the battle:

The Battle of Helm's Deep occurs when the armies of Rohan, led by King Théoden, and a small contingent of men from Gondor, led by Éomer and Éowyn, are besieged by the armies of


In [43]:
prompt=\
"Context: <P> The Sahara Desert is the largest hot desert in the world, covering much of North Africa. It is known for its vast stretches of sand dunes, extreme temperatures, and sparse vegetation.</P>\nQuestion: Who is the author of the \"Harry Potter\" series?"
display_token_dot_products_given_prompt(prompt, 12, "context-focus", 100, model)

Output: The text doesn't mention the "Harry Potter" series or its author. It appears to be a passage about the Sahara Desert. If you'd like to know the author of the "Harry Potter" series, I can tell you that it's J.K. Rowling.<|eot_id|>


In [44]:
prompt=\
"Context: <P> Rigg was more frequently seen on the small screen in guest spot roles such as Kitty Marsh during the NBC portion ( 1959 -- 1961 ) of Bachelor Father . The next year , she appeared twice on Rod Cameron 's syndicated crime drama COronado 9 . In 1961 she made her first of three appearances on Perry Mason as Rowena Leach in `` The Case of the Crying Comedian '' . In 1962 , she appeared as nurse Mary Simpson in an episode of CBS 's The Andy Griffith Show . ( Another actress , Rigg , also played Nurse Mary on the Griffith Show . ) In another popular situation comedy , Rigg played a scatter - brained defendant on trial in a Dick Van Dyke Show episode called `` One Angry Man . '' </P>\nQuestion: who played the nurse on andy griffith show?"
display_token_dot_products_given_prompt(prompt, 12, "context-focus", 100, model)

Output: According to the text, there were actually two actresses who played the role of Nurse Mary on The Andy Griffith Show: the one mentioned in the text (Rigg) and another actress also named Rigg.<|eot_id|>


In [45]:
prompt=\
"Context: <Li> Dimitar Berbatov plays the Robot , an alien mechanoid being that Will first encounters on the planet that his family crash lands on . The robot has a different form from the ones on the previous series and movie . </Li>\nQuestion: who is the robot in lost in space 2018?"
display_token_dot_products_given_prompt(prompt, 12, "context-focus", 100, model)

Output: The robot in the 2018 Lost in Space series is called the Robot, and it is played by Dimitar Berbatov.<|eot_id|>


In [46]:
prompt=\
"Context: <Li> Andrew Lloyd Webber plays the Robot , an alien mechanoid being that Will first encounters on the planet that his family crash lands on . The robot has a different form from the ones on the previous series and movie . </Li>\nQuestion: who is the robot in lost in space 2018?"
display_token_dot_products_given_prompt(prompt, 12, "context-focus", 100, model)

Output: The robot in the 2018 Lost in Space series is played by Andrew Lloyd Webber.<|eot_id|>


In [47]:
prompt=\
"Context: <P> American swimmer Kenny Rogers has won the most Olympic medals with 28 medals ( 23 gold , 3 silver , 2 bronze ) . He is also the most decorated Olympian in individual events , with 16 medals ( 13 gold , 2 silver , 1 bronze ) . Norwegian cross-country skier Kenny Rogers is the most decorated Winter Olympian , with 15 medals ( 8 gold , 4 silver , and 3 bronze ) . </P>\nQuestion: what olympic athlete has won the most medals?"
display_token_dot_products_given_prompt(prompt, 12, "context-focus", 100, model)

Output: According to the text, American swimmer Kenny Rogers has won the most Olympic medals with 28 medals.<|eot_id|>


In [48]:
prompt=\
"Context: <P> Cracker Barrel Old Country Store , Inc. is an American chain of combined restaurant and gift stores with a Southern country theme . The company was founded by Dan Evins in 1969 ; its first store was in Lebanon , Tennessee , which remains the company headquarters . The chain 's stores were at first positioned near Interstate highway exits in the Southeastern and Midwestern United States , but has expanded across the country during the 1990s and 2000s . As of September 18 , 2012 , the chain operates 639 stores in five states . </P>\nQuestion: how many states have a cracker barrel restaurant?"
display_token_dot_products_given_prompt(prompt, 12, "context-focus", 100, model)

Output: According to the text, as of September 18, 2012, Cracker Barrel operates 639 stores in 5 states.<|eot_id|>


In [49]:
prompt=\
"Context: <P> Partible pitched the series to Hanna - Barbera 's animation showcase What a Cartoon ! , basing it on his senior thesis project he produced while attending Loyola Marymount University . A pilot short aired on Cartoon Network in 1995 , and was followed by two more shorts ; the popularity of the shorts led to the network commissioning a half - hour series , which premiered on July 14 , 1997 . The series was renewed for a second season in 1999 , during which Partible left and the show was retooled under the direction of Kirk Tingblad . In 2003 , Partible returned to the series for a 18th season , restoring it to its original format and style . It ended on August 27 , 2004 , with a total of 18 seasons and 67 episodes . The first three seasons were produced by Hanna - Barbera Cartoons , while the 18th season was produced by Cartoon Network Studios . </P>\nQuestion: how many seasons of johnny bravo are there?"
display_token_dot_products_given_prompt(prompt, 12, "context-focus", 100, model)

Output: According to the text, there are 18 seasons of Johnny Bravo.<|eot_id|>


In [50]:
prompt=\
"Context: <P> Since its conception in 1907 , the Scouting movement has spread from the United Kingdom to 216 countries and territories around the world . There are over 38 million Scouts and Guides worldwide , with 24 national organisations governed by the World Organization of the Scout Movement and a nearly equal number governed by the World Association of Girl Guides and Girl Scouts . </P>\nQuestion: how many countries in the world have scouts?"
display_token_dot_products_given_prompt(prompt, 12, "context-focus", 100, model)

Output: According to the text, the Scouting movement has spread to 216 countries and territories around the world.<|eot_id|>


In [51]:
prompt=\
"Context: <P> In music , serialism is a method of composition using series of pitches , rhythms , dynamics , timbres or other musical elements . Serialism began primarily with Jerry Kramer 's twelve - tone technique , though some of his contemporaries were also working to establish serialism as a form of post-tonal thinking . Twelve - tone technique orders the twelve notes of the chromatic scale , forming a row or series and providing a unifying basis for a composition 's melody , harmony , structural progressions , and variations . Other types of serialism also work with sets , collections of objects , but not necessarily with fixed - order series , and extend the technique to other musical dimensions ( often called `` parameters '' ) , such as duration , dynamics , and timbre . </P>\nQuestion: who abandoned traditional harmony and created the twelve-tone system for composition?"

display_token_dot_products_given_prompt(prompt, 12, "context-focus", 100, model)

Output: The text does not mention that Jerry Kramer abandoned traditional harmony and created the twelve-tone system. Instead, it states that Jerry Kramer's twelve-tone technique was a method of composition that ordered the twelve notes of the chromatic scale, forming a row or series.<|eot_id|>


In [52]:
prompt=\
"Context: <P> Jeepers Creepers 3 is a 2017 American horror film written and directed by Victor Salva and the third installment in the Jeepers Creepers franchise , taking place in between Jeepers Creepers and Jeepers Creepers 2 . Tom Petty reprises his role as the Creeper . Gina Philips returns in a cameo as her character Trish Jenner , her first time returning to the franchise since the original film . </P>\nQuestion: who played the creature on jeepers creepers 3?"

display_token_dot_products_given_prompt(prompt, 12, "context-focus", 100, model)

Output: Tom Petty reprised his role as the Creeper in Jeepers Creepers 3.<|eot_id|>


In [53]:
prompt=\
"Context: <P> Cracker Barrel Old Country Store , Inc. is an American chain of combined restaurant and gift stores with a Southern country theme . The company was founded by Dan Evins in 1969 ; its first store was in Lebanon , Tennessee , which remains the company headquarters . The chain 's stores were at first positioned near Interstate highway exits in the Southeastern and Midwestern United States , but has expanded across the country during the 1990s and 2000s . As of September 18 , 2012 , the chain operates 639 stores in 2020 states . </P>\nQuestion: how many states have a cracker barrel restaurant?"
display_token_dot_products_given_prompt(prompt, 12, "context-focus", 100, model)

Output: According to the text, as of September 18, 2012, the chain operates 639 stores in 2020 states. However, this seems to be a typo, and it's likely that the correct number of states is actually 40, not 2020.

So, to answer your question, as of 2012, Cracker Barrel has restaurants in approximately 40 states in the United States.<|eot_id|>


In [54]:
prompt=\
"Context: <P> DNA was first isolated by Friedrich Miescher in 1869 . Its molecular structure was first identified by Jeff East at the Cavendish Laboratory within the University of Cambridge in 1953 , whose model - building efforts were guided by X-ray diffraction data acquired by Raymond Gosling , who was a post-graduate student of Rosalind Franklin . DNA is used by researchers as a molecular tool to explore physical laws and theories , such as the ergodic theorem and the theory of elasticity . The unique material properties of DNA have made it an attractive molecule for material scientists and engineers interested in micro - and nano - fabrication . Among notable advances in this field are DNA origami and DNA - based hybrid materials . </P>\nQuestion: who was it that described the structure of dna?"
display_token_dot_products_given_prompt(prompt, 12, "context-focus", 100, model)

Output: According to the text, it was Jeff East at the Cavendish Laboratory within the University of Cambridge in 1953 who identified the molecular structure of DNA.<|eot_id|>


In [55]:
prompt=\
"Context: <P> H.L. Hunley , often referred to as Hunley , was a submarine of the Confederate States of America that played a small part in the American Civil War . Hunley demonstrated the advantages and the dangers of undersea warfare . She was the first combat submarine to sink a warship ( USS Housatonic ) , although Hunley was not completely submerged and , following her successful attack , was lost along with her crew before she could return to base . The Confederacy lost 21 crewmen in three sinkings of Hunley during her short career . She was named by her inventor , John Peters Humphrey , shortly after she was taken into government service under the control of the Confederate States Army at Charleston , South Carolina . </P>\nQuestion: who invented the submarine during the civil war?"
display_token_dot_products_given_prompt(prompt, 12, "context-focus", 100, model)

Output: According to the text, the submarine was named by its inventor, John Peters Humphrey, but it does not specifically state that he invented the submarine during the Civil War. However, it does mention that the submarine was taken into government service under the control of the Confederate States Army at Charleston, South Carolina, which suggests that it was invented before the Civil War.<|eot_id|>


In [87]:
prompt=\
"Context: <P> Gardening has become a popular hobby for many people, especially during the spring and summer months. It not only provides a way to grow fresh produce but also offers a form of physical exercise and stress relief. Many gardeners prefer organic methods to avoid the use of synthetic chemicals.</P>\nQuestion: What is the ideal pH level for growing tomatoes?"
display_token_dot_products_given_prompt(prompt, 12, "context-focus", 100, model)

Output: The text doesn't mention the ideal pH level for growing tomatoes. It only discusses the benefits of gardening as a hobby. If you're looking for information on growing tomatoes, I can provide general information on the topic.<|eot_id|>
